In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import scipy as sc
import time
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from keras.models import Sequential, load_model
#from keras.layers import LSTM, Dense, Dropout

from translate import Translator

%matplotlib inline

In [2]:
URL = "https://www.booking.com/he"
PATH = "C:/Users/eden4/anaconda/chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get("https://www.booking.com/he")
time.sleep(5)

In [3]:
def load_soup_object(link):
    link = requests.get(link,headers=0)
    soup = BeautifulSoup(link.content, "html.parser")
    return soup

In [6]:
Hotel_name = []
Staff_rate = []
Facilities_rate = []
Cleanliness_rate = []
Comfort_rate = []
Value_for_money_rate = []
Location_rate = []
WiFi_rate = []
import urllib.request

htmlsource = urllib.request.FancyURLopener({"http":"http://127.0.0.1:8080"}).open(URL).read().decode("utf-8")
soup1 = load_soup_object(URL)
searches = soup1.find(class_="ia_section-container clearfix wcu-seo-regions-poi").find_all('a')
#searches is 
cities = ["אזור ירושלים","אזור המרכז","גליל","גליל עליון","כנרת","אזור תל אביב","דרום הארץ","ים המלח","נגב","צפון הארץ","רמת הגולן"]
names = []
indexes = []
href_list = []
counter = 0
for i in searches:
    name1 = i.get_text().replace("\n", "")
    if name1 in cities:
        names.append(name1)
        indexes.append(counter)
        href = searches[counter].get("href").strip()
        URL2=URL+href
        soup2=load_soup_object(URL2)
        searches2 = soup2.find(class_="lp-bui-section bui-spacer--largest x2").find_all("a")
        print("region is:", name1)
        nexthref = URL + searches2[0].get("href").strip()
        href_list.append(nexthref)
        print("href:\n", nexthref)
    counter+=1

C:\Users\eden4\AppData\Local\Temp/ipykernel_8080/3175742329.py:11: DeprecationWarning: FancyURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  htmlsource = urllib.request.FancyURLopener({"http":"http://127.0.0.1:8080"}).open(URL).read().decode("utf-8")


ConnectionError: HTTPSConnectionPool(host='www.booking.com', port=443): Max retries exceeded with url: /he (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F6E9F497C0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [20]:
counter2 = 0
number = 1
offset_flag = 25
for link in href_list: #i.e north, south, negev
    driver.get(link)   
    city_soup = load_soup_object(link)
    number_of_pages = int(city_soup.find(class_="d493c719bc").find_all('li')[-1].get_text())
    hotels = city_soup.find(class_="d4924c9e74").find_all('h3') # find all links(names links) for all hotels in the page into a list.
    print("len hotels : ", len(hotels))
    for number in range(number_of_pages):
        link = link + "offset="+str(offset_flag)
    # now we start extracting information for the dataframe. we start by taking the hotel names into the hotel name list
        for item in hotels: # looping through the h3 class for the hotels --- item == href of single hotel
            flag = 0
            print("hi2")
            print("hotel : ",item.a.find(class_="fcab3ed991 a23c043802").get_text())
            hotel_eng_name = ""
            #if the hotel name is in hebrew - we will now fetch the english name for it - straight from the URL
            # because the hotel name starts after the 33rd character in the URL
            item_for_name = item.a.get("href").strip()[33:]
            for s in item_for_name:
                if s == ".":
                    break
                hotel_eng_name += s
            Hotel_name.append(hotel_eng_name)
            print("hotel in english : ", hotel_eng_name)
            print("link is :",item.a.get("href").strip())
            hotel_soup = load_soup_object(item.a.get("href").strip())
        
        
        
            #now lets get inside the link of the current hotel - and fetch the information for our DataFrame
            hotel_props = hotel_soup.select(".v2_review-scores_body.v2_review-scores_body--compared_to_average li")
            hotel_props2 = hotel_props[14:]
            print("hotel props len is : ", len(hotel_props2))        
            if (len(hotel_props2)) < 7:
                print("problem in hotel : ", Hotel_name[counter2])
                print("hotel props: ", hotel_props2)
                flag = 1
            if flag == 1:
                Staff_rate.append(0)
                Facilities_rate.append(0)
                Cleanliness_rate.append(0)
                Comfort_rate.append(0)
                Value_for_money_rate.append(0)
                Location_rate.append(0)
                WiFi_rate.append(0)
            else:
                Staff_rate.append(hotel_props2[0].find(class_="c-score-bar__score").get_text())
                Facilities_rate.append(hotel_props2[1].find(class_="c-score-bar__score").get_text())
                Cleanliness_rate.append(hotel_props2[2].find(class_="c-score-bar__score").get_text())
                Comfort_rate.append(hotel_props2[3].find(class_="c-score-bar__score").get_text())
                Value_for_money_rate.append(hotel_props2[4].find(class_="c-score-bar__score").get_text())
                Location_rate.append(hotel_props2[5].find(class_="c-score-bar__score").get_text())
                WiFi_rate.append(hotel_props2[6].find(class_="c-score-bar__score").get_text())

            
            
            print("Hotel_Names:", len(Hotel_name),"\n",Hotel_name)
            print("Staff_Rating",len(Staff_rate),"\n",Staff_rate)
            print("Facilities_Rating",len(Facilities_rate),"\n",Facilities_rate)
            print("Cleanliness_Rating",len(Cleanliness_rate),"\n",Cleanliness_rate)
            print("Comfort_Rating",len(Comfort_rate),"\n",Comfort_rate)
            print("Value_For_Money_Rating",len(Value_for_money_rate),"\n",Value_for_money_rate)
            print("Location_Rating",len(Location_rate),"\n",Location_rate)
            print("Free_WiFi_Rating",len(WiFi_rate),"\n",WiFi_rate)
            
            print("\n\n\n")
            print("page number is : ", number)
            
        #offset_flag += 25
        driver.get(link)


        
    dfdict = {"Staff_Rating":Staff_rate, "Facilities_Rating":Facilities_rate, "Cleanliness_Rating":Cleanliness_rate, "Comfort_Rate":Comfort_rate, "Value_For_Money_Rate":Value_for_money_rate, "Location_Rate":Location_rate, "Free_WiFi_Rating":WiFi_rate,"Hotel_Names":Hotel_name}
    df = pd.DataFrame(data=dfdict)
    print(df)
    break

len hotels :  25
hi2
hotel :  Desert chilling - סתלבט במדבר
hotel in english :  stlbt-bmdbr
link is : https://www.booking.com/hotel/il/stlbt-bmdbr.he.html?label=gen173nr-1FCAEoggI46AdIDlgEaGqIAQGYAQ64AQrIAQXYAQHoAQH4AQ6IAgGoAgO4Avj2lJQGwAIB0gIkZmUxZGY3NTUtOTJiMy00MzcyLWJkYzEtY2ZiYzZmYWIxMWY52AIG4AIB&sid=e50b33a8d1d2d0c30beee9bf772a05ea&aid=304142&ucfs=1&arphpl=1&dest_id=3230&dest_type=region&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=cfd3848111860268&srepoch=1652899843&from=searchresults#hotelTmpl
hotel props len is :  7
Hotel_Names: 30 
 ['xxxx-xc-x-xx', 'the-green-guesthouse', 'dead-sea-club-guest-house', 'zimer-dora', 'leonardo-plaza-dead-sea', 'lesanel-guest-house', 'dead-sea-yehuda-apartments', 'khvvh-ykhydt-yrvkh', 'oasis-spa-club-dead-sea-hotel', 'isrotel-dead-sea', 'new-the-luxe-suite-in-arad', 'dy-bmdbr', 'rom-hatayelet', 'vastu-dead-sea-apartment', 'sweet-home-rd', 'guest-house-tzimer-lana', 'belfer-s-dead-

hotel props len is :  7


TypeError: 'str' object is not callable

In [18]:
df.style

,Staff_Rating,Facilities_Rating,Cleanliness_Rating,Comfort_Rate,Value_For_Money_Rate,Location_Rate,Free_WiFi_Rating,Hotel_Names
0,8.5,6.6,7.7,7.6,7.1,8.3,7.6,xxxx-xc-x-xx
1,9.6,8.9,9.6,9.2,8.7,9.4,9.3,the-green-guesthouse
2,9.5,9.4,9.6,9.2,9.3,8.9,8.8,dead-sea-club-guest-house
3,8.1,7.6,8.0,7.7,7.8,8.0,9.0,zimer-dora
4,8.3,8.0,8.1,8.2,7.2,8.7,7.7,leonardo-plaza-dead-sea
5,8.4,8.2,8.9,8.3,8.0,8.4,10,lesanel-guest-house
6,0,0,0,0,0,0,0,dead-sea-yehuda-apartments
7,9.9,9.4,9.5,9.4,9.3,9.2,9.7,khvvh-ykhydt-yrvkh
8,8.0,7.6,7.7,7.8,7.1,8.6,7.5,oasis-spa-club-dead-sea-hotel
9,8.3,8.4,8.5,8.5,7.5,8.8,8.7,isrotel-dead-sea
